In [11]:
import json
from openai import OpenAI
import os
import re
import httpx

In [13]:
# key for sorting
def natural_keys(text):
    return [int(c) if c.isdigit() else c.lower() for c in re.split('([0-9]+)', text)]


In [159]:
#Creates the format for the jsonl file
def create_jsonl_format(subdomain, prompt, id, nl=False):
    jsonl = []
    for idx, (domain, prompt) in enumerate(zip(subdomain, prompt)):
        start_index = domain.find("(define")
        dom = domain[start_index:]
        if nl:
            ids = id+str(idx)+"_nl"
        else:
            ids = id+str(idx)
        jsonl.append({"custom_id": ids, "method":"POST", "url":"/v1/chat/completions", "body":{"model": "gpt-4o", "messages": [{"role": "system", "content": "You are a PDDL domain creation expert, you modify domains based on requested changes."},{"role": "user", "content": f"{prompt}\n\n{dom}"}],"max_tokens": 3012, "temperature": 1, "top_p": 1}})
    return jsonl

In [177]:
# For passing directly no batching
def create_jsonl_format2(subdomain, prompt, id, nl=False):
    jsonl = []
    for idx, (domain, prompt) in enumerate(zip(subdomain, prompt)):
        start_index = domain.find("(define")
        dom = domain[start_index:]
        if nl:
            ids = id+str(idx)+"_nl"
        else:
            ids = id+str(idx)
        jsonl.append({"custom_id": ids, "messages": [{"role": "system", "content": "You are a PDDL domain creation expert, you modify domains based on requested changes."},{"role": "user", "content": f"{prompt}\n\n{dom}"}]})
    return jsonl

In [ ]:
# Code and natural language
#Creates the format for the jsonl file
def create_jsonl_format_code(subdomain, code, id, nl=None):
    jsonl = []
    for idx, (domain, prompt) in enumerate(zip(subdomain, prompt)):
        start_index = domain.find("(define")
        dom = domain[start_index:]
        if nl is not None:
            ids = id+str(idx)+"_nl"
            jsonl.append({"custom_id": ids, "method":"POST", "url":"/v1/chat/completions", "body":{"model": "gpt-4o", "messages": [{"role": "system", "content": "You are a PDDL domain creation expert, you modify provided domains to make them executable based on a given prompt and python wrapper"},{"role": "user", "content": f"{nl}\n\n{dom}\n\nThe following is the executable code for the domain:\n\n{code}"}],"max_tokens": 4512, "temperature": 1, "top_p": 1}})
        else:
            ids = id+str(idx)
            if id != 'Si' and id != 'Ti':
                jsonl.append({"custom_id": ids, "method":"POST", "url":"/v1/chat/completions", "body":{"model": "gpt-4o", "messages": [{"role": "system", "content": "You are a PDDL domain creation expert, you modify provided domains to make them executable based on the codebase."},{"role": "user", "content": f"Modify the following domain based on the given code so that all the functions in the code match the actions of the domain taking into effect preconditions and effects.\n\n{dom}\n\nThe following is the executable code for the domain:\n\n{code}"}],"max_tokens": 4512, "temperature": 1, "top_p": 1}})
            else:
                jsonl.append({"custom_id": ids, "method":"POST", "url":"/v1/chat/completions", "body":{"model": "gpt-4o", "messages": [{"role": "system", "content": "You are a PDDL domain creation expert, you modify provided domains to make them executable based on the codebase."},{"role": "user", "content": f"Modify the following domain based on the given code so that all the functions in the code match the actions of the domain taking into effect preconditions and effects. All durative actions have a duration of 4\n\n{dom}\n\nThe following is the executable code for the domain:\n\n{code}"}],"max_tokens": 4512, "temperature": 1, "top_p": 1}})

    return jsonl

In [211]:
#Creates the Batch for passing to openai
def create_jsonl_batch(root, format_function, batch_type="prompt-only"):
    path = os.path.abspath(root)
    batch_json = []
    for domain in ["Depots", "DriverLog", "Rovers"]:
        for req in ["Numeric", "SimpleTime", "Strips", "Time"]:
            prompt_files = os.listdir(os.path.join(path,domain,req, "labels"))
            nl_prompts = sorted([prompts for prompts in prompt_files if prompts.endswith("nl.txt")], key=natural_keys)
            reg_prompts = sorted([prompts for prompts in prompt_files if not prompts.endswith("nl.txt")], key=natural_keys)
            
            subdomain_files = os.listdir(os.path.join(path,domain,req, "subdomains"))
            subdomains = [subdomain for subdomain in subdomain_files if "sub" in subdomain]
            subdomains = sorted(subdomains, key=natural_keys)
            
            if batch_type == "prompt-and-code":
                with open(os.path.join(path,domain,req, "signature.py"), "r") as f:
                    code = f.read()

            sub_prompt = []
            for subdomain in subdomains:
                with open(os.path.join(path,domain,req, "subdomains", subdomain), "r") as f:
                    sub_prompt.append(f.read())
            
            nl_pr = []
            for prompt in nl_prompts:
                with open(os.path.join(path,domain,req, "labels", prompt), "r") as f:
                    nl_pr.append(f.read())
                    
            reg_pr = []
            for prompt in reg_prompts:
                with open(os.path.join(path,domain,req, "labels", prompt), "r") as f:
                    reg_pr.append(f.read())

            print("creating jsonl file for ", domain, req)
            if batch_type == "prompt-only":
                nl_jsonl = format_function(nl_pr, f"{domain}_{req[:2]}", True)
                reg_jsonl = format_function(reg_pr, f"{domain}_{req[:2]}")
                
            elif batch_type == "prompt-and-code":
                nl_jsonl = format_function(sub_prompt, code, f"{domain}_{req[:2]}_code", nl_pr)
                reg_jsonl = format_function(sub_prompt, code, f"{domain}_{req[:2]}_code") 
                
            batch_json.extend(reg_jsonl)
            batch_json.extend(nl_jsonl)

    with open(f"results/batch-{batch_type}.jsonl", "w") as file:
        for line in batch_json:
            file.write(json.dumps(line))
            file.write("\n")

In [205]:
#Function to upload batch data to openai
def save_batch_info(name, data):
    with open("batch_info.json", "w") as file:
        json.dump({name: data}, file)
def upload_batch(client, jsonl_file):
    batch_file_input = client.files.create(
        file=open(jsonl_file, "rb"),
        purpose="batch"
    )
    #save_batch_info("openai_batch_info", batch_file_input)
    print("Batch file uploaded successfully")
    return batch_file_input 

In [206]:
#Function to start inference on the batch
def start_run(client, input_file_id, description = "Evaluation run with no code"):
    run = client.batches.create(
        input_file_id=input_file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={"description": description}    
    )
    try:
        save_batch_info("openai_run_info", run)
    except:
        print("Error saving run info")
    print("Run started successfully")
    return run

In [212]:
# Create Batch no code
path = "data/IPC3/Tests1"
create_jsonl_batch(path, create_jsonl_format, "prompt-only")

creating jsonl file for  Depots Numeric
creating jsonl file for  Depots SimpleTime
creating jsonl file for  Depots Strips
creating jsonl file for  Depots Time
creating jsonl file for  DriverLog Numeric
creating jsonl file for  DriverLog SimpleTime
creating jsonl file for  DriverLog Strips
creating jsonl file for  DriverLog Time
creating jsonl file for  Rovers Numeric
creating jsonl file for  Rovers SimpleTime
creating jsonl file for  Rovers Strips
creating jsonl file for  Rovers Time


In [ ]:
# Batch with code
path = "data/IPC3/Tests1"
create_jsonl_batch(path, create_jsonl_format_code, "prompt-and-code")

In [208]:
with open("EvaluationPipeline/OPENAI_KEY.json", 'r') as json_file:
        data = json.load(json_file)
        client = OpenAI(api_key=data['openai_key'])

In [213]:
with open("results/batch-prompt-only.jsonl", "r") as file:
    batch = file.readlines()
    batch = [json.loads(line) for line in batch]
for i in range(5):
    print(batch[i])

{'custom_id': 'Depots_Nu0_nl', 'method': 'POST', 'url': '/v1/chat/completions', 'body': {'model': 'gpt-4o', 'messages': [{'role': 'system', 'content': 'You are a PDDL domain creation expert, you modify domains based on requested changes.'}, {'role': 'user', 'content': 'Modify the following PDDL domain by \nadding a load_limit to the truck. \n\nThis will require adding 3 functions: \nload_limit per truck, current_load per truck and the weight per crate\n\nModify the load and unload actions accordingly\n\n\n(define (domain Depot)\n(:requirements :typing :fluents)\n(:types place locatable - object\n\tdepot distributor - place\n        truck hoist surface - locatable\n        pallet crate - surface)\n\n(:predicates (at ?x - locatable ?y - place) \n             (on ?x - crate ?y - surface)\n             (in ?x - crate ?y - truck)\n             (lifting ?x - hoist ?y - crate)\n             (available ?x - hoist)\n             (clear ?x - surface)\n)\n\n(:functions  \n\t(fuel-cost)\n)\n\t\n(:

In [ ]:
with open("results/batch-prompt-and-code.jsonl", "r") as file:
    batch = file.readlines()
    batch = [json.loads(line) for line in batch]
for i in range(5):
    print(batch[i])

In [214]:
#Upload batch data to openai
batch_file_input = upload_batch(client, "batch-prompt-only.jsonl")
#batch_file_input = upload_batch(client, "batch-prompt-and-code.jsonl")

Batch file uploaded successfully


In [215]:
batch_file_input.id

'file-uzwrJEXlYGj0QDx3DwDO3LRH'

In [216]:
batch_info = start_run(client, batch_file_input.id)

Error saving run info
Run started successfully


In [217]:
# Check batch ID
#print(batch_info)
print(batch_info.id)
batch_id = batch_info.id

batch_hagBjDRgG8WTL42gWLpjPf2b


In [225]:
# check current status of the run
status = client.batches.retrieve(batch_id)
print(f"Status: {status.status}\nOutput_id: {status.output_file_id}\nBatchRequestsCount: {status.request_counts}")

Status: completed
Output_id: file-GiAX7eLHZ4Lb5SoqmPi92zBz
BatchRequestsCount: BatchRequestCounts(completed=270, failed=0, total=270)


In [226]:
#Getting results fromt the run
content = client.files.content(status.output_file_id)
file_contents = content.content
contents = file_contents.decode("utf-8")
json_content = []
for line in contents.split("\n"):
    if line:
        json_content.append(json.loads(line))

In [230]:
def save_results(json_content, name = "results.jsonl"):
    with open(f"results/{name}", "w") as file:
            for line in json_content:
                file.write(json.dumps(line))
                file.write("\n")

In [ ]:
save_results(json_content)
#save_results(json_content, "results_code.jsonl")

In [231]:
#Extract domains
def extract_domain(jsonl_file):
    with open(jsonl_file, "r") as file:
        jsonl = file.readlines()
        jsonl = [json.loads(line) for line in jsonl]
    domains = {}
    for line in jsonl:
            domain = line["response"]["body"]["choices"][0]["message"]["content"].split("```")[1]
            domain = domain[domain.index("("):]
            tag = line["custom_id"]
            domains[tag] = domain
    return domains

In [232]:
domains = extract_domain("results.jsonl")
#domains = extract_domain("results_code.jsonl")

In [233]:
print(domains.keys())

dict_keys(['Depots_Nu0_nl', 'Depots_Nu1_nl', 'Depots_Nu2_nl', 'Depots_Nu3_nl', 'Depots_Nu4_nl', 'Depots_Nu5_nl', 'Depots_Nu6_nl', 'Depots_Nu7_nl', 'Depots_Nu8_nl', 'Depots_Nu9_nl', 'Depots_Nu10_nl', 'Depots_Nu11_nl', 'Depots_Nu0', 'Depots_Nu1', 'Depots_Nu2', 'Depots_Nu3', 'Depots_Nu4', 'Depots_Nu5', 'Depots_Nu6', 'Depots_Nu7', 'Depots_Nu8', 'Depots_Nu9', 'Depots_Nu10', 'Depots_Nu11', 'Depots_Si0_nl', 'Depots_Si1_nl', 'Depots_Si2_nl', 'Depots_Si3_nl', 'Depots_Si4_nl', 'Depots_Si5_nl', 'Depots_Si6_nl', 'Depots_Si7_nl', 'Depots_Si8_nl', 'Depots_Si9_nl', 'Depots_Si10_nl', 'Depots_Si0', 'Depots_Si1', 'Depots_Si2', 'Depots_Si3', 'Depots_Si4', 'Depots_Si5', 'Depots_Si6', 'Depots_Si7', 'Depots_Si8', 'Depots_Si9', 'Depots_Si10', 'Depots_St0_nl', 'Depots_St1_nl', 'Depots_St2_nl', 'Depots_St3_nl', 'Depots_St4_nl', 'Depots_St5_nl', 'Depots_St6_nl', 'Depots_St7_nl', 'Depots_St0', 'Depots_St1', 'Depots_St2', 'Depots_St3', 'Depots_St4', 'Depots_St5', 'Depots_St6', 'Depots_St7', 'Depots_Ti0_nl', 'Depo

In [234]:
#Save PDDL results
def save_pddl(domains, path, subfolder="AI_gen"):
    for key, value in domains.items():
        if key.startswith("Depots"):
            if "Nu" in key:
                with open(os.path.join(path, "Depots", "Numeric", subfolder, f"{key}.pddl"), "w") as file:
                    file.write(value)
            elif "Si" in key:
                with open(os.path.join(path, "Depots", "SimpleTime", subfolder, f"{key}.pddl"), "w") as file:
                    file.write(value)
            elif "St" in key:
                with open(os.path.join(path, "Depots", "Strips", subfolder, f"{key}.pddl"), "w") as file:
                    file.write(value)
            elif "Ti" in key:
                with open(os.path.join(path, "Depots", "Time", subfolder, f"{key}.pddl"), "w") as file:
                    file.write(value)
        elif key.startswith("DriverLog"):
            if "Nu" in key:
                with open(os.path.join(path, "DriverLog", "Numeric", subfolder, f"{key}.pddl"), "w") as file:
                    file.write(value)
            elif "Si" in key:
                with open(os.path.join(path, "DriverLog", "SimpleTime", subfolder, f"{key}.pddl"), "w") as file:
                    file.write(value)
            elif "St" in key:
                with open(os.path.join(path, "DriverLog", "Strips", subfolder, f"{key}.pddl"), "w") as file:
                    file.write(value)
            elif "Ti" in key:
                with open(os.path.join(path, "DriverLog", "Time", subfolder, f"{key}.pddl"), "w") as file:
                    file.write(value)
        elif key.startswith("Rovers"):
            if "Nu" in key:
                with open(os.path.join(path, "Rovers", "Numeric", subfolder, f"{key}.pddl"), "w") as file:
                    file.write(value)
            elif "Si" in key:
                with open(os.path.join(path, "Rovers", "SimpleTime", subfolder, f"{key}.pddl"), "w") as file:
                    file.write(value)
            elif "St" in key:
                with open(os.path.join(path, "Rovers", "Strips", subfolder, f"{key}.pddl"), "w") as file:
                    file.write(value)
            elif "Ti" in key:
                with open(os.path.join(path, "Rovers", "Time", subfolder, f"{key}.pddl"), "w") as file:
                    file.write(value)
    print(f"PDDL files saved successfully in {path}")

In [235]:
path = "data/IPC3/Tests1"
path = os.path.abspath(path)
save_pddl(domains, path)
#save_pddl(domains, path, "AI_gen_code")

PDDL files saved successfully in /home/god/Documents/Masters/data/IPC3/Tests1


# Report file
## Syntax
- Domain
    - Requirement
        - Total syntax errors
        - Syntax errors natural language
        - Syntax errors regular
        - total success

## Plan
- Domain
    - Requirement
        - Failed plan generations natural language
        - Failed plan generations regular
        - Total failed plan generations
        - Total success

In [2]:
#Run in python to get outputs
import json
import os
import subprocess
subprocess.os.chdir("/home/god/Documents/Masters")

syntax_path = "results/syntax-AI_gen.jsonl"
#syntax_path = "results/syntax-AI_gen_code.jsonl"

with open(syntax_path, "r") as file:
    syntax_results = file.readlines()
    syntax_results = [json.loads(line) for line in syntax_results]

print(syntax_results[0])

{'domain': 'Depots', 'requirement': 'Numeric', 'file': 'Depots_Nu7.pddl', 'errors': 0, 'warnings': 0, 'error_list': [], 'warning_list': []}


In [3]:
#Process results
Depots = {"Numeric": 0, "SimpleTime": 0, "Strips": 0, "Time": 0}
DriverLog = {"Numeric": 0, "SimpleTime": 0, "Strips": 0, "Time": 0}
Rovers = {"Numeric": 0, "SimpleTime": 0, "Strips": 0, "Time": 0}
syntax_evaluation = {"Depots": Depots, "DriverLog": DriverLog, "Rovers": Rovers}
natural_language_errors = 0
invalid_syntax_files = []
for result in syntax_results:
    if result["errors"] > 0:
        syntax_evaluation[result["domain"]][result["requirement"]] += 1
        invalid_syntax_files.append(result["file"])
        if "_nl" in result["file"]:
            natural_language_errors += 1

total_numerics = sum([syntax_evaluation["Depots"]["Numeric"], syntax_evaluation["DriverLog"]["Numeric"], syntax_evaluation["Rovers"]["Numeric"]])
total_simpletimes = sum([syntax_evaluation["Depots"]["SimpleTime"], syntax_evaluation["DriverLog"]["SimpleTime"], syntax_evaluation["Rovers"]["SimpleTime"]])
total_strips = sum([syntax_evaluation["Depots"]["Strips"], syntax_evaluation["DriverLog"]["Strips"], syntax_evaluation["Rovers"]["Strips"]])
total_time = sum([syntax_evaluation["Depots"]["Time"], syntax_evaluation["DriverLog"]["Time"], syntax_evaluation["Rovers"]["Time"]])

Total_depot = sum([syntax_evaluation["Depots"]["Numeric"], syntax_evaluation["Depots"]["SimpleTime"], syntax_evaluation["Depots"]["Strips"], syntax_evaluation["Depots"]["Time"]])
Total_DriverLog = sum([syntax_evaluation["DriverLog"]["Numeric"], syntax_evaluation["DriverLog"]["SimpleTime"], syntax_evaluation["DriverLog"]["Strips"], syntax_evaluation["DriverLog"]["Time"]])
Total_Rovers = sum([syntax_evaluation["Rovers"]["Numeric"], syntax_evaluation["Rovers"]["SimpleTime"], syntax_evaluation["Rovers"]["Strips"], syntax_evaluation["Rovers"]["Time"]])

Total = sum([Total_depot, Total_DriverLog, Total_Rovers])

percentage_errors = "{:.2f}".format((Total/270)*100)

regular_expression_errors = Total - natural_language_errors
regular_expression_percentage = "{:.2f}".format((regular_expression_errors/Total)*100)
natural_language_errors = "{:.2f}".format((natural_language_errors/Total)*100)
        


In [4]:
print(f"Total errors: {Total}")
print(f"Total errors percentage: {percentage_errors}%")
print(f"Successful modification without automated corrections: {'{:.2f}'.format(100-float(percentage_errors))}%\n")
print(f"Total numeric errors: {total_numerics}")
print(f"Total simpletime errors: {total_simpletimes}")
print(f"Total strips errors: {total_strips}")
print(f"Total time errors: {total_time}\n")

print(f"Total Depot errors: {Total_depot}")
print(f"Total DriverLog errors: {Total_DriverLog}")
print(f"Total Rovers errors: {Total_Rovers}\n")

print(f"Total regular expression errors: {regular_expression_errors}")
print(f"Regular expression errors percentage: {regular_expression_percentage}%")
print(f"Natural language errors: {natural_language_errors}%")
print(f"Total natural language errors: {natural_language_errors}\n\n")

print("Files with invalid syntax", invalid_syntax_files)

Total errors: 11
Total errors percentage: 4.07%
Successful modification without automated corrections: 95.93%

Total numeric errors: 0
Total simpletime errors: 5
Total strips errors: 1
Total time errors: 5

Total Depot errors: 1
Total DriverLog errors: 4
Total Rovers errors: 6

Total regular expression errors: 7
Regular expression errors percentage: 63.64%
Natural language errors: 36.36%
Total natural language errors: 36.36


Files with invalid syntax ['Depots_Si7.pddl', 'DriverLog_Si11.pddl', 'DriverLog_Si10.pddl', 'DriverLog_Si3_nl.pddl', 'DriverLog_Ti8.pddl', 'Rovers_Si1_nl.pddl', 'Rovers_St8.pddl', 'Rovers_Ti5.pddl', 'Rovers_Ti2_nl.pddl', 'Rovers_Ti6_nl.pddl', 'Rovers_Ti6.pddl']


In [14]:
import subprocess
import concurrent.futures

pddl_path = "/home/god/Documents/Masters/data/IPC3/Tests1"
popf_path = "/home/god/.planutils/packages/popf"
domain_folder = "AI_gen"

Depots = {"Numeric": 0, "SimpleTime": 0, "Strips": 0, "Time": 0}
DriverLog = {"Numeric": 0, "SimpleTime": 0, "Strips": 0, "Time": 0}
Rovers = {"Numeric": 0, "SimpleTime": 0, "Strips": 0, "Time": 0}
plan_evaluation = {"Depots": Depots, "DriverLog": DriverLog, "Rovers": Rovers}
natural_language_errors = 0
files_with_failed_plans = []
files_timeout = []

runs = []
subprocess.os.chdir(popf_path)
total = 0
success = 0
for domain in ["Depots", "DriverLog", "Rovers"]:
    for req in ["Numeric", "SimpleTime", "Strips", "Time"]:
        files = sorted(os.listdir(os.path.join(pddl_path, domain, req, domain_folder)), key=natural_keys)
        for file in files:
            if file.endswith(".pddl") and file not in invalid_syntax_files:
                problem_files = sorted(os.listdir(os.path.join(pddl_path, domain, req, "problems")), key=natural_keys)
                for problem in problem_files:
                    plan_command = f"./run /home/god/Documents/Masters/data/IPC3/Tests1/{domain}/{req}/{domain_folder}/{file} /home/god/Documents/Masters/data/IPC3/Tests1/{domain}/{req}/problems/{problem}"
                    runs.append(plan_command)
                    # try:
                    #     plan = subprocess.run(plan_command, shell=True, capture_output=True, check=True, timeout=5)
                    #     out = plan.stdout
                    #     total += 1
                    #     if "Solution Found" in out.decode("utf-8"):
                    #         success += 1
                    #     else:
                    #         files_with_failed_plans.append(file)
                    #         plan_evaluation[domain][req] += 1
                    #         if "_nl" in file:
                    #             natural_language_errors += 1
                    # except subprocess.TimeoutExpired:
                    #     files_timeout.append(file)
                    #     print("Timeout error")
                    # except subprocess.CalledProcessError:
                    #     print("Error generating plan")
  

# Total_plan_depot = sum([plan_evaluation["Depots"]["Numeric"], plan_evaluation["Depots"]["SimpleTime"], plan_evaluation["Depots"]["Strips"], plan_evaluation["Depots"]["Time"]])
# Total_plan_DriverLog = sum([plan_evaluation["DriverLog"]["Numeric"], plan_evaluation["DriverLog"]["SimpleTime"], plan_evaluation["DriverLog"]["Strips"], plan_evaluation["DriverLog"]["Time"]])
# Total_plan_Rovers = sum([plan_evaluation["Rovers"]["Numeric"], plan_evaluation["Rovers"]["SimpleTime"], plan_evaluation["Rovers"]["Strips"], plan_evaluation["Rovers"]["Time"]])

# Total_plan = sum([Total_plan_depot, Total_plan_DriverLog, Total_plan_Rovers])

# Total_plan_numerics = sum([plan_evaluation["Depots"]["Numeric"], plan_evaluation["DriverLog"]["Numeric"], plan_evaluation["Rovers"]["Numeric"]])
# Total_plan_simpletimes = sum([plan_evaluation["Depots"]["SimpleTime"], plan_evaluation["DriverLog"]["SimpleTime"], plan_evaluation["Rovers"]["SimpleTime"]])
# Total_plan_strips = sum([plan_evaluation["Depots"]["Strips"], plan_evaluation["DriverLog"]["Strips"], plan_evaluation["Rovers"]["Strips"]])
# Total_plan_time = sum([plan_evaluation["Depots"]["Time"], plan_evaluation["DriverLog"]["Time"], plan_evaluation["Rovers"]["Time"]])

# percentage_errors = "{:.2f}".format((Total_plan/total)*100)
# percentage_success = "{:.2f}".format((success/total)*100)

# Total_plan_nl_errors = natural_language_errors
# Total_plan_regular_errors = Total_plan - natural_language_errors
# Total_plan_regular_errors_percentage = "{:.2f}".format((Total_plan_regular_errors/Total_plan)*100)
# Total_plan_nl_errors_percentage = "{:.2f}".format((Total_plan_nl_errors/Total_plan)*100)
subprocess.os.chdir("/home/god/Documents/Masters/runs")
with open("run_commands.txt", "w") as file:
    for run in runs:
        file.write(run)
        file.write("\n")

In [299]:
print(f"Total plan errors: {Total_plan}")
print(f"Total plan errors percentage: {percentage_errors}%")
print(f"Total plan success: {percentage_success}%\n")

print(f"Total plan numeric errors: {Total_plan_numerics}")
print(f"Total plan simpletime errors: {Total_plan_simpletimes}")
print(f"Total plan strips errors: {Total_plan_strips}")
print(f"Total plan time errors: {Total_plan_time}\n")

print(f"Total plan Depot errors: {Total_plan_depot}")
print(f"Total plan DriverLog errors: {Total_plan_DriverLog}")
print(f"Total plan Rovers errors: {Total_plan_Rovers}\n")

print(f"Total regular expression errors: {Total_plan_regular_errors}")
print(f"Regular expression errors percentage: {Total_plan_regular_errors_percentage}%")
print(f"Natural language errors: {Total_plan_nl_errors}")
print(f"Natural language errors percentage: {Total_plan_nl_errors_percentage}%\n\n")

print("Files with failed plans", files_with_failed_plans)

NameError: name 'Total_plan' is not defined